# Introduction to this "tutorial"

This notebook is an interactive example for a "simple" **staircase sweep measurments experiment**, it can be loosely mapped to the Programming Example in pp-198 of B1500A mannual.

Experimental configurations are:

**Active SMUs: Terminals of MOSFET:**
* SMU1 - **channel 1** - HPSMU: **Drain**  
* SMU2 - **channel 3** - HPSMU: **Gate**
* GND - channel xx - GNDU: **Source** 
    * (GNDU is not explicitly declared either in Pymeausure or Programming Manual, here we treat it as a **"dont care"**)

## 1. Setup environment

In [18]:
import pymeasure
from pymeasure.adapters import VXI11Adapter
from pymeasure.adapters import VISAAdapter
from pymeasure.instruments import Instrument
# from pymeasure.instruments.agilent import AgilentB1500
from SourceCodes.agilent1505 import b1505a
import time

## 2. Connect Instrument via adapter

In [19]:
# adapter = VXI11Adapter("TCPIP::10.8.128.178::gpib0,17::INSTR")

# Though VISA adapter throws warning messages. It works fine for this experiment
adapter = VISAAdapter("TCPIP::10.8.128.178::gpib0,17::INSTR")

instr = b1505a(adapter, timeout='None')
instr.reset()

* Query activate SMU modules being used

In [20]:
module_names = instr.query_modules()
print(f'SMUs used: {module_names}')

SMUs used: {1: 'HPSMU', 3: 'HPSMU', 5: 'MFCMU', 6: 'HCSMU', 8: 'HVSMU'}


* Initialize and Enable these SMUs
* Enable filters
* Use HRADC if possible

In [21]:
instr.initialize_all_smus()
highres_adc = True
for i, smu in enumerate(instr.smu_references, start=1):
    smu.enable() # enable SMUs
    print(f'smu{i} stands for channel {smu.channel}')
    smu.filter = 1
    if highres_adc:
        if module_names[smu.channel] == 'HCSMU' or module_names[smu.channel] == 'HVSMU'\
            or module_names[smu.channel] == 'MCSMU': # Programming Mannual pp-353
            pass
        else:
            smu.adc_type = 'HRADC'
    smu.meas_op_mode = 'VOLTAGE'
instr.data_format(21, mode=1)

smu1 stands for channel 1
smu2 stands for channel 3
smu3 stands for channel 6
smu4 stands for channel 8


In [22]:
print(instr.smu_references)

dict_values([<SourceCodes.agilent1505.rerSMU object at 0x7fc3e9dca910>, <SourceCodes.agilent1505.rerSMU object at 0x7fc3ed886550>, <SourceCodes.agilent1505.rerSMU object at 0x7fc3ed857390>, <SourceCodes.agilent1505.rerSMU object at 0x7fc3ed8185d0>])


In [23]:
instr.smu3.disable() # Disable SMUs we dont use
instr.smu4.disable() # Disable SMUs we dont use

drain = instr.smu1
gate = instr.smu2 
source = instr.GNDU  

In [24]:
instr.adc_setup('HRADC','MANUAL', 6)

## Setup sampling (Stress sampling)
Let's set sample point to 10 in this simple example

In [25]:
t = 60
nop = 10
interval = t // nop
nchannels = len(instr.smu_references)

icomp = 0.1
vg = 3
vd = 3

```sampling_timing``` mapped to ```MT```

In [26]:
instr.meas_mode('SAMPLING', *instr.smu_references)
nchannels = len(instr.smu_references)

arugment:  (<SourceCodes.agilent1505.rerSMU object at 0x7fc3e9dca910>, <SourceCodes.agilent1505.rerSMU object at 0x7fc3ed886550>, <SourceCodes.agilent1505.rerSMU object at 0x7fc3ed857390>, <SourceCodes.agilent1505.rerSMU object at 0x7fc3ed8185d0>)


In [27]:
instr.sampling_mode = 'LINEAR'
instr.sampling_timing(0.1, 2, nop) ## sampling_timing(h_bias, interval, nop, h_base)
instr.sampling_auto_abort(False,post='BIAS') #MSC: BASE/BIAS
instr.time_stamp = True

In [28]:
drain.sampling_source('VOLTAGE', 'Auto Ranging', vd, vd, icomp)
gate.sampling_source('VOLTAGE', 'Auto Ranging', vg, vg, icomp)

In [29]:
#Start Measurement
instr.check_errors()
instr.clear_buffer()
instr.clear_timer()
instr.send_trigger()

In [30]:
# instr.time_stamp = True

In [31]:
meas = []
for i in range(nop):
    read_data = instr.read_channels(1+2*nchannels) #Sampling Index + (time stamp + measurement value) * number of channels
    # process live data for plotting etc.
    # data format for every channel (status code, channel name e.g. 'SMU1', data name e.g 'Current Measurement (A)', value)
    meas.append(read_data)

In [32]:
print(meas)

[(('000', 'MISC', 'Sampling index', 1), ('000', 'SMU1', 'Time (s)', 0.21365), ('000', 'SMU1', 'Voltage Measurement (V)', 2.9996), ('000', 'SMU2', 'Time (s)', 0.21563), ('000', 'SMU2', 'Voltage Measurement (V)', 3.0), ('000', 'SMU3', 'Time (s)', 0.21712), ('000', 'SMU3', 'Voltage Measurement (V)', 0.0), ('000', 'SMU4', 'Time (s)', 0.21804), ('000', 'SMU4', 'Voltage Measurement (V)', 0.0028)), (('000', 'MISC', 'Sampling index', 2), ('000', 'SMU1', 'Time (s)', 2.21409), ('000', 'SMU1', 'Voltage Measurement (V)', 2.9996), ('000', 'SMU2', 'Time (s)', 2.2163), ('000', 'SMU2', 'Voltage Measurement (V)', 3.0), ('000', 'SMU3', 'Time (s)', 2.21769), ('000', 'SMU3', 'Voltage Measurement (V)', 4e-05), ('000', 'SMU4', 'Time (s)', 2.21853), ('000', 'SMU4', 'Voltage Measurement (V)', -0.0082)), (('000', 'MISC', 'Sampling index', 3), ('000', 'SMU1', 'Time (s)', 4.21404), ('000', 'SMU1', 'Voltage Measurement (V)', 2.9992), ('000', 'SMU2', 'Time (s)', 4.21583), ('000', 'SMU2', 'Voltage Measurement (V)',

In [33]:
print(len(meas))

10


In [34]:
for e in meas:
    print(e)

(('000', 'MISC', 'Sampling index', 1), ('000', 'SMU1', 'Time (s)', 0.21365), ('000', 'SMU1', 'Voltage Measurement (V)', 2.9996), ('000', 'SMU2', 'Time (s)', 0.21563), ('000', 'SMU2', 'Voltage Measurement (V)', 3.0), ('000', 'SMU3', 'Time (s)', 0.21712), ('000', 'SMU3', 'Voltage Measurement (V)', 0.0), ('000', 'SMU4', 'Time (s)', 0.21804), ('000', 'SMU4', 'Voltage Measurement (V)', 0.0028))
(('000', 'MISC', 'Sampling index', 2), ('000', 'SMU1', 'Time (s)', 2.21409), ('000', 'SMU1', 'Voltage Measurement (V)', 2.9996), ('000', 'SMU2', 'Time (s)', 2.2163), ('000', 'SMU2', 'Voltage Measurement (V)', 3.0), ('000', 'SMU3', 'Time (s)', 2.21769), ('000', 'SMU3', 'Voltage Measurement (V)', 4e-05), ('000', 'SMU4', 'Time (s)', 2.21853), ('000', 'SMU4', 'Voltage Measurement (V)', -0.0082))
(('000', 'MISC', 'Sampling index', 3), ('000', 'SMU1', 'Time (s)', 4.21404), ('000', 'SMU1', 'Voltage Measurement (V)', 2.9992), ('000', 'SMU2', 'Time (s)', 4.21583), ('000', 'SMU2', 'Voltage Measurement (V)', 2.